In [1]:
import os
os.environ['PYSPARK_PYTHON'] = "C:/Users/user/anaconda3/envs/torchenv/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/user/anaconda3/envs/torchenv/python.exe"

### Spark RDD
- Spark 1.x, Spark 2.x에서의 전톧적인 RDD 
- `Sparkconf()`와 `SparkContext()`를 통합한 방법이 DataFrame으로 생성하는 `SparkSession()`

### SparkContext
- Spark 애플리케이션의 **entry point** 역할로, Spark 클러스터와 연결하고 RDD(Resilient Distributed Dataset)를 생성하거나 브로드캐스트 변수 등을 관리하며 Spark 작업 환경을 설정하고 제어하는 데 사용
- `SparkContext` 자체는 데이터를 담고 있지 않음

In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('FreiendsByAge')
sc = SparkContext(conf = conf) 
sc

<SparkContext master=local appName=FreiendsByAge>

### 나이별 친구 수 추출

In [3]:
def parse_line(line):
  fields = line.split(",")
  age = int(fields[2])
  num_friends = int(fields[3])
  
  return (age, num_friends)

In [4]:
lines = sc.textFile("file:///nvidia_course/nvidia-course/week1/day5/SparkRDD/data/fakefriends.csv")
lines

file:///nvidia_course/nvidia-course/week1/day5/SparkRDD/data/fakefriends.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
rdd = lines.map(parse_line)
rdd

PythonRDD[2] at RDD at PythonRDD.scala:53

- `mapValues()`: RDD의 값(value) 부분만 변환
- `reduceByKey()`: 동일한 키를 가진 값들을 그룹화하여 람다 함수를 이용해 병합

In [11]:
# value 값만 변환하므로 만약 데이터 (33, 385)이라면, (33, (385 , 1))로 반환
# 33세의 평균 친구의 수를 구하기 위해 몇 명의 33세가 있는지도 고려해야함
# reduceByKey를 활용하여 각 33세의 친구 수와 33세의 등장 횟수를 각각 키를 기준으로 계산
totals_by_age = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [12]:
avg_by_age = totals_by_age.mapValues(lambda x: x[0] / x[1])

### DAG로 Spark가 최적의 방법을 찾아서 작동
- `collect()`: PySpark에서 액션(Action)변환으로, DAG(Direted Acyclic Graph)와 지연 평가(Lazy Evaluation)라는 Spark와 관련이 있음
  - RDD 또는 DataFrame의 모든 데이터를 Spark 클러스터의 드라이버 프로그램으로 가져오는 액션으로, 분산되어 있는 데이터를 한 곳으로 모으는 역할
  - OOM(Out of Memory) 문제를 일으키거나 네트워크 부하 가능성 O

- `지연 평가(lazy Evaluation)`
  - Spark의 모든 변환(Transformation)(ex. map, filter, reduceByKey)은 모두 지연적으로 평가
  - 변환 코드를 작성해서 Spark는 즉시 연산을 수행하지 않고 실행 계획만 세워둔 후, 실제 연산은 액션(Action)(ex. collect(), count(), show(), write)이 호출될 때 실행됨

- `DAG(Directed Acyclic Graph)`: Spark는 지연 평가 방식을 통해 DAG 실행 모델 구축
  - Directed(방향성): 작업 흐름이 한 방향으로만 진행되어 되돌아가거나 순환 X 
  - Acyclic(비순환): 작업 그래프에 순환 X
  - Graph(그래프): 일련의 노드(Spark, DataFrame)와 엣지(Transformation)로 구성된 집합

`collect()`가 호출되면:

1. Spark는 collect()까지의 전체 변환 체인을 분석하여 DAG를 생성

2. DAG 스케줄러는 이 DAG를 기반으로 스테이지(Stage) 를 나누는데, 스테이지는 셔플(shuffle) 같은 넓은 변환(Wide Transformation)을 기준으로 나뉨 

3. 각 스테이지 내에서 태스크(Task) 들이 생성되고, 이 태스크들이 클러스터의 워커 노드에서 병렬로 실행

4. 모든 태스크가 완료되면, collect()는 최종 결과를 드라이버 프로그램으로 다시 모음.

In [13]:
results = avg_by_age.collect()
for res in results:
  print(res)

(33, 325.3333333333333)
(26, 242.05882352941177)
(55, 295.53846153846155)
(40, 250.8235294117647)
(68, 269.6)
(59, 220.0)
(37, 249.33333333333334)
(54, 278.0769230769231)
(38, 193.53333333333333)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(56, 306.6666666666667)
(43, 230.57142857142858)
(36, 246.6)
(22, 206.42857142857142)
(35, 211.625)
(45, 309.53846153846155)
(60, 202.71428571428572)
(67, 214.625)
(19, 213.27272727272728)
(30, 235.8181818181818)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(42, 303.5)
(49, 184.66666666666666)
(48, 281.4)
(50, 254.6)
(39, 169.28571428571428)
(32, 207.9090909090909)
(58, 116.54545454545455)
(64, 281.3333333333333)
(31, 267.25)
(52, 340.6363636363636)
(24, 233.8)
(20, 165.0)
(62, 220.76923076923077)
(41, 268.55555555555554)
(44, 282.1666666666667)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(28, 209.1)
(66, 276.44444444444446)
(46, 223.69230769230768)
(29, 215.91666666666666)
(18, 343.375)
(47, 233.22222222222